# Prepare BATADAL dataset for Numenta Benchmark (NAB)

The [Numenta Benchmark](https://github.com/numenta/NAB) consists of multiple anomaly detection algorithms for time-series. The algorithms identify anomalies contextually - based on previous values for that time series. Therefore the NAB repository expects each time series to be in its own CSV with timestamp and value as the columns.

This notebook converts the BATADAL CSV into multiple CSVs, one for each system variable. The three BATADAL datasets are not contiguous, and since NAB is contextual, we only use the test BATADAL dataset.

In [31]:
import pandas as pd
from collections import defaultdict
from typing import List, Tuple, Dict
from pathlib import Path

In [2]:
import json

In [3]:
train_no_anom = pd.read_csv("../../data/01_raw/BATADAL_dataset03_train_no_anomaly.csv")
train_some_anom = pd.read_csv("../../data/01_raw/BATADAL_dataset04_train_some_anomaly.csv")
test_with_anom = pd.read_csv("../../data/01_raw/BATADAL_test_dataset_some_anomaly.csv")

In [4]:
# has leading white space
train_some_anom.columns = train_some_anom.columns.str.strip()

In [5]:
train_no_anom.shape, train_some_anom.shape, test_with_anom.shape

((8761, 45), (4177, 45), (2089, 44))

# NAB
NAB expects a directly containing CSVs with `timestamp, value` as the columns. Therefore we will split each CSV into multiple CSVs as required

In [6]:
sensor_cols = [c for c in test_with_anom.columns if c not in ["DATETIME", "ATT_FLAG", "timestamp"]]

## Split Test Set

As NAB finds anomalies at individual time series level by using previous time steps to predict future time steps, we only need the BATADAL test set as it is 3-months of continuous data with anomalies.

In [21]:
def convert_str_datetime(anomalies: List[Tuple]) -> List[Tuple]:
    fmt ="%d/%m/%Y %H"
    anomalies_dt = [
        (pd.to_datetime(s, format=fmt), pd.to_datetime(e, format=fmt)) for s, e in anomalies
    ]
    
    return anomalies_dt

In [27]:
def append_anomaly_column(anomalies: List[Tuple], df: pd.DataFrame) -> pd.DataFrame:        
    df["timestamp"] = pd.to_datetime(df["DATETIME"], format="%d/%m/%y %H")
    df = df.set_index(["timestamp"])

    df["attack"] = 0
    for start, end in anomalies:
        df.loc[start:end, "attack"] = 1
        
    return df

In [24]:
def split_csv_and_save(
    test: pd.DataFrame,
    test_anomalies: List[Tuple],
    sensor_columns: List[str], 
    parameters: Dict
) -> None:
    csv_save_dir = Path(parameters["ts_data_dir"])
    label_save_dir = Path(parameters["ts_label_dir"])
    csv_save_dir.mkdir(parents=True, exist_ok=True)
    label_save_dir.mkdir(parents=True, exist_ok=True)
    
    anomaly_dict = defaultdict(list)
    for c in sensor_columns:
        test.reset_index()[["timestamp", c]].rename(
            columns={c:"value"}
        ).to_csv(f"{csv_save_dir}/{c}.csv", index=False)

        for s_anom, e_anom in test_anomalies:
            anomaly_dict[f"{c}.csv"].append([
                s_anom.strftime('%Y-%m-%d %H:%M:%S.%f'), e_anom.strftime('%Y-%m-%d %H:%M:%S.%f')
            ])
            
    with open(f"{label_save_dir}/labels-test.json", "w") as fp:
        json.dump(anomaly_dict, fp, indent=4)

### Append anomaly column

In [25]:
# http://www.batadal.net/images/Attacks_TestDataset.png
test_anomalies = [
    ("16/01/2017 09", "19/01/2017 06"),
    ("30/01/2017 08", "02/02/2017 00"),
    ("09/02/2017 03", "10/02/2017 09"),
    ("12/02/2017 01", "13/02/2017 07"),
    ("24/02/2017 05", "28/02/2017 08"),
    ("10/03/2017 14", "13/03/2017 21"),
    ("25/03/2017 20", "27/03/2017 01")
]

test_anomalies_dt = convert_str_datetime(test_anomalies)

In [28]:
test_df = append_anomaly_column(test_anomalies_dt, test_with_anom)
test_df.shape

(2089, 45)

### Save splitted CSVs

In [29]:
parameters = {
    "ts_data_dir": "../../data/02_intermediate/iot/ts_data",
    "ts_label_dir": "../../data/02_intermediate/iot/ts_label"
    
}

In [32]:
split_csv_and_save(test_df, test_anomalies_dt, sensor_cols, parameters)

In [33]:
!ls "../../data/02_intermediate/iot/ts_data"

F_PU10.csv  F_PU6.csv  L_T3.csv    P_J269.csv  P_J317.csv  S_PU3.csv  S_V2.csv
F_PU11.csv  F_PU7.csv  L_T4.csv    P_J280.csv  P_J415.csv  S_PU4.csv
F_PU1.csv   F_PU8.csv  L_T5.csv    P_J289.csv  P_J422.csv  S_PU5.csv
F_PU2.csv   F_PU9.csv  L_T6.csv    P_J300.csv  S_PU10.csv  S_PU6.csv
F_PU3.csv   F_V2.csv   L_T7.csv    P_J302.csv  S_PU11.csv  S_PU7.csv
F_PU4.csv   L_T1.csv   P_J14.csv   P_J306.csv  S_PU1.csv   S_PU8.csv
F_PU5.csv   L_T2.csv   P_J256.csv  P_J307.csv  S_PU2.csv   S_PU9.csv


In [34]:
!ls "../../data/02_intermediate/iot/ts_label"

labels-combined.json  labels-test.json
